# Projeto xo-mosquito

> Analise exploratoria de infraestrura dos Bairros de Recife

## Proposta
Levando em consideração o contexto das doenças tropicais negligenciadas, em
especial Dengue, Zika e Chikungunya, e seu impacto social e, segundo IANPHI (2016),
econômico na américa latina, especialmente no Brasil, o projeto possui como proposta:
- Analisar a relação entre o perfil social dos indivíduos afetados e as incidências de doenças transmitidas pelo mosquito Aedes Aegypti;
- Investigar a influência da localidade física e climáticos em que se encontram tais indivíduos;
- Encontrar padrões nos casos relatados para assim sugerir abordagens de vigilância que possam ser implementados por governos.

O objetivo é analisar essas hipóteses e tentar validá-las ou não em relação a Recife,
Pernambuco, mas se não for possível, devido a falta de dados, então o escopo será analisar
globalmente.

## 1. Coleta e limpeza dos dados

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

Dataset fornecido pela Secretaria de Infraestrutura e Serviços Urbanos de Recife pelo site http://dados.recife.pe.gov.br.

[__Área urbana__](http://dados.recife.pe.gov.br/dataset/area-urbana)

Este conjunto descreve dados sobre a área espacial da cidade do Recife, incluindo dados sobre divisões do espaço físico da cidade, como: bairros, distritos, setores, quadras, faces de quadra, RPAs (Regiões Político Administrativas), micro-regiões, praças, parques, áreas verdes, recursos hídricos, lotes e edificações.

### 1.1 Bairros
Identifica os bairros do Recife, com a localização geográfica e a associação com a RPA (Região Político Administrativa) e a Microregião onde o bairro está inserido.

In [2]:
bairros_resumido = pd.read_csv('http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/ddcf8195-23c0-4ecc-94b5-ec6533a5ca74/download/tabela-de-bairros.csv', encoding = 'latin1', sep=';')

print(bairros_resumido.head(5))

geojson = pd.read_json('http://dados.recife.pe.gov.br/dataset/c1f100f0-f56f-4dd4-9dcc-1aa4da28798a/resource/e43bee60-9448-4d3d-92ff-2378bc3b5b00/download/bairros.geojson')

   Nº Localidade       Nome Localidade Nome Município
0            132               AFLITOS         RECIFE
1            779              AFOGADOS         RECIFE
2            337             AGUA FRIA         RECIFE
3            493         ALTO DO MANDU         RECIFE
4            540   ALTO JOSE BONIFACIO         RECIFE


In [3]:
bairros = json_normalize(geojson.features)
bairros.drop(['type','geometry.type','geometry.coordinates'], axis=1, inplace=True)
bairros = bairros.set_index('id')
bairros.rename(index=str, columns={"properties.bairro_codigo": "codigo", "properties.bairro_nome": "nome", "properties.bairro_nome_ca": "nome_caixa_alta", "properties.microrregiao": "microrregiao", "properties.rpa": "rpa"}, inplace=True)
bairros

,codigo,nome,nome_caixa_alta,microrregiao,rpa
id,,,,,
0,728,Cidade Universitária,CIDADE UNIVERSITARIA,3,4
1,116,Soledade,SOLEDADE,2,1
2,710,Engenho do Meio,ENGENHO DO MEIO,2,4
3,841,Caçote,CACOTE,2,5
4,884,Cohab,COHAB,3,6
5,736,Várzea,VARZEA,3,4
6,701,Torrões,TORROES,2,4
7,698,Iputinga,IPUTINGA,1,4
8,752,Curado,CURADO,1,5


In [4]:
bairros['microrregiao'] = bairros.microrregiao.astype('category')
bairros['rpa'] = bairros.rpa.astype('category')
bairros['nome'] = bairros.nome.astype('str')
bairros['nome_caixa_alta'] = bairros.nome_caixa_alta.astype('str')
bairros.dtypes

codigo                int64
nome                 object
nome_caixa_alta      object
microrregiao       category
rpa                category
dtype: object

### 1.2 Recursos Hídricos

Identifica geograficamente a bacia hidrográfica da cidade do Recife, contendo rios, braços de rios, riachos, bacias portuárias, canais, córregos, açudes, lagoas, alagados na cidade do Recife.

In [5]:
geojson = pd.read_json('data/recursoshidricos/RecursosHidricos.geojson')

In [6]:
hidricos = json_normalize(geojson.features)
hidricos.drop(['type','geometry.type','geometry.coordinates'],axis=1, inplace=True)
hidricos = hidricos.set_index('id')
hidricos.rename(index=str, columns={"properties.nome_recursohidrico": "nome"}, inplace=True)
hidricos.head(5)

,nome
id,
0,RIO CAPIBARIBE
1,RIO CAPIBARIBE
2,RIO CAPIBARIBE
3,RIO CAPIBARIBE
4,RIO CAPIBARIBE


Sem informações para cruzar com os bairros

## 1.3 Logradouros
Este dataset contém a relação de todos os logradouros da cidade de Recife, bem como a definição dos campos dos serviços oferecidos pela prefeitura por face de logradouro.
 - Dataset = http://dados.recife.pe.gov.br/dataset/area-urbana/resource/01187dc5-61d0-438b-b1bc-b1ec508a3e0f
 - Metadados = http://dados.recife.pe.gov.br/dataset/area-urbana/resource/a1ac6fa9-03f6-4843-9f4e-83a658824332

In [7]:
logradouros = pd.read_csv('data/logradouros.csv',sep=';')
logradouros.head(5)

,distrito,quadra,face,codlogradouro,tipologradouro,nomelogradouro,codbairro,nomebairro,valor_v0,codiluminacao,...,desccoleta,codarborizacao,codredeagua,codlimpezaurbana,desclimpezaurbana,codredesgoto,codredetelefonica,codguiasesarjetas,codgaleriapluviais,codredeeletrica
0,1,20,4,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
1,1,40,3,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
2,1,45,4,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
3,1,50,4,388,RUA,Rua General Abreu e Lima,183,ROSARINHO,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1
4,1,70,2,388,RUA,Rua General Abreu e Lima,191,TAMARINEIRA,30.0,3,...,CONVENCIONAL ALTERNA,1,1,2,REGULAR ALTERNADA,1,1,1,1,1


In [8]:
# converte tipos para categorico
logradouros['tipologradouro'] = logradouros.tipologradouro.astype('category')
logradouros['desciluminacao'] = logradouros.desciluminacao.astype('category')
logradouros['descpavimentacao'] = logradouros.descpavimentacao.astype('category')
logradouros['desccoleta'] = logradouros.desccoleta.astype('category')
logradouros['codarborizacao'] = logradouros.codarborizacao.astype('category')
logradouros['codredeagua'] = logradouros.codredeagua.astype('category')
logradouros['desclimpezaurbana'] = logradouros.desclimpezaurbana.astype('category')
logradouros['codredesgoto'] = logradouros.codredesgoto.astype('category')
logradouros['codredetelefonica'] = logradouros.codredetelefonica.astype('category')
logradouros['codguiasesarjetas'] = logradouros.codguiasesarjetas.astype('category')
logradouros['codgaleriapluviais'] = logradouros.codgaleriapluviais.astype('category')
logradouros['codredeeletrica'] = logradouros.codredeeletrica.astype('category')

# ajuste de alguns valores
logradouros['codarborizacao'] = logradouros['codarborizacao'].map({2: 0, 1: 1})
logradouros['codredeagua'] = logradouros['codredeagua'].map({2: 0, 1: 1})
logradouros['codredesgoto'] = logradouros['codredesgoto'].map({2: 0, 1: 1})
logradouros['codredetelefonica'] = logradouros['codredetelefonica'].map({2: 0, 1: 1})
logradouros['codguiasesarjetas'] = logradouros['codguiasesarjetas'].map({2: 0, 1: 1})
logradouros['codgaleriapluviais'] = logradouros['codgaleriapluviais'].map({2: 0, 1: 1})
logradouros['codredeeletrica'] = logradouros['codredeeletrica'].map({2: 0, 1: 1})

# drop colunas desnecessarias ou reduntantes
logradouros.drop(['face','valor_v0','codiluminacao','codpavimentacao', 'codcoleta','codlimpezaurbana'], axis=1, inplace=True)

# renomeia algumas colunas por conveniencia
logradouros = logradouros.rename(index=str,columns={"desciluminacao": "iluminacao","descpavimentacao":"pavimentacao",\
                                                    "desccoleta":"coleta", "codarborizacao":"arborizacao",\
                                                    "codredeagua":"redeagua", "desclimpezaurbana":"limpezaurbana",\
                                                    "codredetelefonica":"redetelefonica","codguiasesarjetas":"guiasesarjetas",\
                                                    "codgaleriapluviais":"galeriapluviais","codredeeletrica":"redeeletrica"})

In [9]:
logradouros.groupby('nomebairro')['codlogradouro'].count().describe()

count      45.000000
mean      462.644444
std       591.400885
min         1.000000
25%       127.000000
50%       324.000000
75%       507.000000
max      2640.000000
Name: codlogradouro, dtype: float64

In [19]:
casos_dengue = pd.read_csv('data/casos-dengue2015.csv',sep=';',low_memory=False)

In [20]:
casos_dengue_logradouro = pd.read_csv('logadouro.csv',low_memory=False)
casos_dengue_logradouro = casos_dengue_logradouro.rename(index=str, columns={"nome_logradouro_residencia": "nome"})

Intersecção dos datasets

In [21]:
import unidecode

logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.upper())
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: unidecode.unidecode(x))
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.replace("AVENIDA", "AV"))
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.replace("AV.", "AV"))
logradouros.nomelogradouro = logradouros.nomelogradouro.apply(lambda x: x.replace("RUA", "R"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("AVENIDA", "AV"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("AV.", "AV"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("RUA", "R"))

#print(casos_dengue_logadouro.head(5))
#logradouros[logradouros.nomelogradouro == "AVENIDA CENTRAL"]

print(casos_dengue_logradouro.head(5))
print(logradouros.nomelogradouro.head(5))

   Unnamed: 0              nome  quantidade
0           0  R ODETE MONTEIRO          35
1           1        AV CENTRAL          29
2           2   R SAO SEBASTIAO          27
3           3    R MASSARANDUBA          26
4           4        R TAPIREMA          25
0    R GENERAL ABREU E LIMA
1    R GENERAL ABREU E LIMA
2    R GENERAL ABREU E LIMA
3    R GENERAL ABREU E LIMA
4    R GENERAL ABREU E LIMA
Name: nomelogradouro, dtype: object


In [34]:
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("1?", "1A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("2?", "2A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("3?", "3A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("4?", "4A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("5?", "5A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("6?", "6A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("7?", "7A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("8?", "8A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("9?", "9A"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("1¬", "1A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("2¬", "2A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("3¬", "3A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("4¬", "4A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("5¬", "5A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("6¬", "6A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("7¬", "7A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("8¬", "8A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("9¬", "9A"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JOS?", "JOSE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("F?", "FE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("CAXANG?", "CAXANGA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("CAXANG?", "CAXANGA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("D?O", "DAO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("Ã?", "CA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?O", "AO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("S?", "SA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("L?N", "LAN"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("D'?", "D'A"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MAGAL?ES", "MAGALHAES"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MAGALH?ES", "MAGALHAES"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?ES", "AES"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("R?GO", "REGO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("PARA?SO", "PARAISO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("ANDR?", "ANDRE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JACA?NA", "JACAUNA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("IRAJ?", "IRAJA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("QUIT?RIA", "QUITERIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("L?RIOS", "LIRIOS"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("ABATI?", "ABATIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("PAJE?", "PAJEU"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("ITABORA?", "ITABORAI"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("FEL?CIO", "FELICIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MAUR?CIO", "MAURICIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JIQUI?", "JIQUIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("FABR?CIO", "FABRICIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("TOM?", "TOME"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("C?LIA", "CELIA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("OT?VIO", "OTAVIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("GEN?SIO", "GENESIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("TI?MA", "TIUMA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("B?RBARA", "BARBARA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("PATROC?NIO","PATROCINIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("LU?S","LUIS"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?GUA", "AGUA"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("APTROC?NIO", "APTROCINIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JERUSAL?M","JERUSALEM"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("?MPAR", "IMPAR"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("SA?DE", "SAUDE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("L?RIO", "LIRIO"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("JOSU?", "JOSUE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("MACA?", "MACAE"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("TR?S", "TRES"))
casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: x.replace("HEMET?RIO", "HEMETERIO"))

casos_dengue_logradouro.nome = casos_dengue_logradouro.nome.apply(lambda x: unidecode.unidecode(x))

In [35]:
casos_dengue_logradouro = casos_dengue_logradouro.drop(casos_dengue_logradouro[casos_dengue_logradouro.nome.str.contains('?', regex=False)].index)
#logradouros[logradouros.nomelogradouro.str.contains('W. R.', regex=False)]
casos_dengue_logradouro

,Unnamed: 0,nome,quantidade
0,0,R ODETE MONTEIRO,35
1,1,AV CENTRAL,29
2,2,R SAO SEBASTIAO,27
3,3,R MASSARANDUBA,26
4,4,R TAPIREMA,25
5,5,R ZEFERINO AGRA,24
6,6,R TORRES HOMEM,24
7,7,R BAMBU,22
8,8,R ENGENHO NORUEGA,21
9,9,R VINTE E UM DE ABRIL,21


In [79]:
logradouros["CASOS_DENGUE"] = np.nan

In [81]:
#qtd = logradouros[]
#candidatos_2014 = candidatos_2014.assign(QTD_CAPITAL = qtd.capital)

#logradouros["CASOS_DENGUE"] = logradouros["CASOS_DENGUE"].apply(lambda x: x + 1)

#if(have_info[i]):
        #logradouros[i].CASOS_DENGUE = casos_dengue_logradouro[casos_dengue_logradouro.nome == logradouros[i].nomelogradouro].quantidade.values[0]

have_info = logradouros.nomelogradouro.isin(casos_dengue_logradouro.nome.tolist()).tolist()
for i in range(0,len(have_info)-1):
    if have_info[i]:
        logradouros.CASOS_DENGUE.iloc[i] = casos_dengue_logradouro[casos_dengue_logradouro.nome == logradouros.iloc[i].nomelogradouro].quantidade.values[0]     

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [91]:
lograd_com_dengue = logradouros.dropna()
print("Mediana dos casos de dengue: {}".format(lograd_com_dengue.CASOS_DENGUE.median()))
lograd_com_dengue.CASOS_DENGUE.describe()

Mediana dos casos de dengue: 2.0


count    6682.000000
mean        4.046992
std         5.434665
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        29.000000
Name: CASOS_DENGUE, dtype: float64

In [96]:
lograd_com_dengue['area_risco'] = lograd_com_dengue.CASOS_DENGUE.map(lambda x: x>2 and 1 or 0)
lograd_com_dengue.area_risco.value_counts()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    4103
1    2579
Name: area_risco, dtype: int64

Conjunto de treino e de teste

In [94]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split

In [102]:
treino, teste = train_test_split(lograd_com_dengue, test_size=0.2)

print(treino.shape)
print(treino.area_risco.value_counts())
print(teste.shape)
print(teste.area_risco.value_counts())

lograd_com_dengue

(5345, 20)
0    3301
1    2044
Name: area_risco, dtype: int64
(1337, 20)
0    802
1    535
Name: area_risco, dtype: int64


,distrito,quadra,codlogradouro,tipologradouro,nomelogradouro,codbairro,nomebairro,iluminacao,pavimentacao,coleta,arborizacao,redeagua,limpezaurbana,codredesgoto,redetelefonica,guiasesarjetas,galeriapluviais,redeeletrica,CASOS_DENGUE,area_risco
19,1,10,13269,RUA,R CAIO PEREIRA,175,ENCRUZILHADA,VAPOR MERCURIO,PARALELO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,2.0,0
20,1,15,13269,RUA,R CAIO PEREIRA,175,ENCRUZILHADA,VAPOR MERCURIO,PARALELO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,2.0,0
21,1,30,13269,RUA,R CAIO PEREIRA,183,ROSARINHO,VAPOR MERCURIO,PARALELO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,2.0,0
22,1,35,13269,RUA,R CAIO PEREIRA,183,ROSARINHO,VAPOR MERCURIO,PARALELO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,2.0,0
23,1,50,13269,RUA,R CAIO PEREIRA,183,ROSARINHO,VAPOR MERCURIO,PARALELO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,2.0,0
24,1,95,25879,RUA,R DO FUTURO,191,TAMARINEIRA,VAPOR MERCURIO,ASFALTO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,0,1,1,6.0,1
29,1,15,36196,RUA,R DOUTOR JOSE MARIA,175,ENCRUZILHADA,VAPOR MERCURIO,ASFALTO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,1.0,0
30,1,16,36196,RUA,R DOUTOR JOSE MARIA,175,ENCRUZILHADA,VAPOR MERCURIO,ASFALTO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,1.0,0
31,1,35,36196,RUA,R DOUTOR JOSE MARIA,183,ROSARINHO,VAPOR MERCURIO,ASFALTO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,1.0,0
32,1,60,36196,RUA,R DOUTOR JOSE MARIA,191,TAMARINEIRA,VAPOR MERCURIO,ASFALTO,CONVENCIONAL ALTERNA,1,1,REGULAR ALTERNADA,1,1,1,1,1,1.0,0


In [120]:
# + coleta + arborizacao + redeagua + limpezaurbana + codredesgoto + guiasesarjetas + galeriapluviais
treinoCP = sm.add_constant(treino.copy())
model = smf.logit("area_risco ~ pavimentacao + coleta + arborizacao + redeagua + limpezaurbana + codredesgoto + guiasesarjetas + galeriapluviais",treinoCP).fit(maxiter=10000, method='nm')

Optimization terminated successfully.
         Current function value: 0.629171
         Iterations: 5027
         Function evaluations: 6380


In [122]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             area_risco   No. Observations:                 5345
Model:                          Logit   Df Residuals:                     5326
Method:                           MLE   Df Model:                           18
Date:                Sun, 01 Jul 2018   Pseudo R-squ.:                 0.05421
Time:                        20:53:58   Log-Likelihood:                -3362.9
converged:                       True   LL-Null:                       -3555.7
                                        LLR p-value:                 9.595e-71
========================================================================================================
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                0.0100      0.162      0.062      0.951      -0.307       0.327
pavimentacao[T.CONCRETO]                 0.3067      0.259      1.182      0.237      -0.202       0.815
pavimentacao[T.ESCADARIA]                0.6031      0.381      1.582      0.114      -0.144       1.350
pavimentacao[T.OUTROS]                   0.1963      2.247      0.087      0.930      -4.207       4.600
pavimentacao[T.PARALELO]                -0.4478      0.107     -4.165      0.000      -0.658      -0.237
pavimentacao[T.POLIEDRO]                -0.2003      0.556     -0.360      0.719      -1.290       0.889
pavimentacao[T.SEM PAVIMENTACAO]        -1.0698      0.122     -8.780      0.000      -1.309      -0.831
coleta[T.CONVENCIONAL DIARIA]            0.2094      0.089      2.344      0.019       0.034       0.384
coleta[T.MANUAL ALTERNADA]               0.9748      0.588      1.657      0.098      -0.178       2.128
coleta[T.MANUAL DIARIA]                  0.0074      0.113      0.066      0.948      -0.213       0.228
limpezaurbana[T.PROGRAMADA SEMANAL]      0.3507      0.121      2.890      0.004       0.113       0.589
limpezaurbana[T.REGULAR ALTERNADA]       0.2206      0.104      2.127      0.033       0.017       0.424
limpezaurbana[T.REGULAR DIARIA]         -0.2055      0.118     -1.742      0.082      -0.437       0.026
limpezaurbana[T.SEM LIMPEZA PUBLICA]    -0.3673      0.137     -2.690      0.007      -0.635      -0.100
arborizacao                             -0.4217      0.095     -4.438      0.000      -0.608      -0.235
redeagua                                 0.2689      0.089      3.014      0.003       0.094       0.444
codredesgoto                            -0.9223      0.104     -8.848      0.000      -1.127      -0.718
guiasesarjetas                           0.4839      0.122      3.967      0.000       0.245       0.723
galeriapluviais                         -0.2014      0.125     -1.617      0.106      -0.445       0.043
========================================================================================================
"""

In [132]:
# Foco no p_value
pd.options.display.float_format = '{:.6f}'.format # remove a notação cientifica para facilitar a leitura

print("Features importantes para definir uma rua com alto nivel de dengue:\n")
print(model.pvalues[model.pvalues < 0.05])

print("\n\nFeatures que não importam para definir se uma rua pode ter alta transmissão de dengue:\n")
print(model.pvalues[model.pvalues >= 0.05])



Features importantes para definir uma rua com alto nivel de dengue:

pavimentacao[T.PARALELO]               0.000031
pavimentacao[T.SEM PAVIMENTACAO]       0.000000
coleta[T.CONVENCIONAL DIARIA]          0.019077
limpezaurbana[T.PROGRAMADA SEMANAL]    0.003852
limpezaurbana[T.REGULAR ALTERNADA]     0.033415
limpezaurbana[T.SEM LIMPEZA PUBLICA]   0.007155
arborizacao                            0.000009
redeagua                               0.002577
codredesgoto                           0.000000
guiasesarjetas                         0.000073
dtype: float64


Features que não importam para definir se uma rua pode ter alta transmissão de dengue:

Intercept                         0.950503
pavimentacao[T.CONCRETO]          0.237264
pavimentacao[T.ESCADARIA]         0.113582
pavimentacao[T.OUTROS]            0.930368
pavimentacao[T.POLIEDRO]          0.718698
coleta[T.MANUAL ALTERNADA]        0.097581
coleta[T.MANUAL DIARIA]           0.947745
limpezaurbana[T.REGULAR DIARIA]   0.081583
ga

In [115]:
model.predict(teste).map(lambda x: x > 0.5 and 'alto risco de dengue' or 'baixo risco de dengue')

8267     baixo risco de dengue
3212     baixo risco de dengue
18297    baixo risco de dengue
20042    baixo risco de dengue
13820    baixo risco de dengue
12970    baixo risco de dengue
9170     baixo risco de dengue
1768     baixo risco de dengue
13996    baixo risco de dengue
15052    baixo risco de dengue
10926     alto risco de dengue
13038    baixo risco de dengue
14935     alto risco de dengue
17142    baixo risco de dengue
13965    baixo risco de dengue
1429     baixo risco de dengue
16585    baixo risco de dengue
10254    baixo risco de dengue
8232      alto risco de dengue
17619    baixo risco de dengue
13819    baixo risco de dengue
7396     baixo risco de dengue
18288    baixo risco de dengue
10713    baixo risco de dengue
14444    baixo risco de dengue
17863    baixo risco de dengue
12240     alto risco de dengue
12679    baixo risco de dengue
16463    baixo risco de dengue
13978    baixo risco de dengue
                 ...          
8402     baixo risco de dengue
7401    

In [116]:
resultados = pd.DataFrame(dict(prob=model.predict(teste),
                  pred=model.predict(teste).map(lambda x: x > 0.5 and 1 or 0),
                  real=teste.area_risco))

resultados.head(5)

,pred,prob,real
8267,0,0.240364,0
3212,0,0.136056,0
18297,0,0.258773,0
20042,0,0.363110,0
13820,0,0.258773,0


In [117]:
tabcontingencia = pd.crosstab(resultados.pred,resultados.real).values
tabcontingencia

array([[706, 394],
       [ 96, 141]])

In [118]:
acc = np.sum(tabcontingencia.diagonal())/teste.shape[0]
sen = tabcontingencia[1,1]/(tabcontingencia[1,1]+tabcontingencia[0,1])
esp = tabcontingencia[0,0]/(tabcontingencia[0,0]+tabcontingencia[1,0])

print("acurácia= {:.3f}".format(acc))
print("sensitividade={:.3f}".format(sen))
print("especificidade={:.3f}".format(esp))

acurácia= 0.634
sensitividade=0.264
especificidade=0.880
